In [3]:
import math
import os
import random
import re
import sys
import time
from multiprocessing import Process, Queue
import networkx as nx
from networkx.algorithms.components import connected_components
from networkx.algorithms.connectivity import node_connectivity
from networkx.algorithms.centrality.betweenness import betweenness_centrality
from networkx.algorithms.distance_measures import diameter
import numpy as np
import elp_networks as elpnet
import elp_networks.algorithms as elpalg
import logbook

In [2]:
b = elpnet.Butterfly(8)
g = nx.Graph(b.edges())
node_connectivity(g)

4

In [5]:
def sample_pairs(nodes, num):
    # Sample sources and targets
    sources = random.sample(nodes, num)
    targets = random.sample(nodes, num)
    # Replace self loops
    for i in range(num - 1):
        if sources[i] == targets[i]:
            sources[i], sources[-1] = sources[-1], sources[i]
    if sources[-1] == targets[-1]:
        sources[-1] = random.choice(set(nodes) - set(sources))
    return zip(sources, targets)
    
def expand(g, source, radius):
    if radius == 0:
        return
    # Get all nodes within radius
    ego = set(nx.ego_graph(g, source, radius=radius).nodes())
    # Keep track of the boundary and remove nodes within radius
    boundary = set()
    for v in ego:
        if v == source:
            continue
        boundary |= set(nx.neighbors(g, v))
        g.remove_node(v)
    # Add edges from source to boundary
    for v in boundary:
        if v == source or v in ego:
            continue
        g.add_edge(source, v)
        

In [9]:
pairs = sample_pairs(g.nodes(), 10)
# Calculate connectivity for sampled pairs
connectivities = []
for source, target in pairs:
    expanded = g.copy()
    expand(expanded, source, 2)
    try:
        expand(expanded, target, 2)
    except KeyError:
        # Target was removed in expansion
        target = source
    c = node_connectivity(expanded, source, target)
    connectivities.append(c)
# Calculate statistics
mean = np.mean(connectivities)
std = np.std(connectivities, ddof=1)
se = mean / np.sqrt(len(connectivities))


In [29]:
s, t
nx.node_connectivity(g, s, s)

4

In [7]:
import pandas as pd
pd.read_csv("output/router_targeted/2017-10-15 15:10:43 d101dee 0/stats.csv")

,removed,diameter,size,node_conn0_mean,node_conn0_se,node_conn1_mean,node_conn1_se,node_conn2_mean,node_conn2_se,node_conn3_mean,node_conn3_se,node_conn_pairs,failed,high_betweenness,node_count,rewire_f,butterfly_m,failure_type
0,0,9,0.000000,1.493333,0.121930,123.180000,10.057605,2296.500000,187.508440,2383.480000,194.610327,150,1,7.314439e+06,6474,0.9,7,targeted
1,1,9,1.028986,1.286667,0.105056,77.873333,6.358331,1530.820000,124.990930,2370.573333,193.556502,150,9,4.046683e+06,6474,0.9,7,targeted
2,2,10,1.029724,1.233333,0.100701,45.686667,3.730301,1250.653333,102.115417,2303.980000,188.119179,150,6,4.269638e+06,6474,0.9,7,targeted
3,3,10,1.031655,1.060000,0.086549,25.380000,2.072268,989.413333,80.785260,2090.833333,170.715827,150,28,3.102704e+06,6474,0.9,7,targeted
4,4,10,1.030769,1.033333,0.084371,22.226667,1.814800,672.373333,54.899053,1926.906667,157.331271,150,0,2.654410e+06,6474,0.9,7,targeted
